# GENERAL TITLE

## Short description of the notebook

# General Setup

## Clone Repository & Install Dependencies



In [ ]:
%%shell
REPO=https://github.com/jacopo-raffaeli/portfolio-replica.git
DIR=portfolio-replica

# Clone if needed, else pull latest
if [ ! -d "$DIR" ]; then
  git clone $REPO
else
  cd $DIR
  git pull origin main
  cd ..
fi

# Enter project root
cd $DIR

# Install all dependencies
pip install -r requirements.txt
